# Specifying targets for a Scene

It is possible to create a 'scene' populated with specified astronomical sources as an input to MIRISim.  The types of targets, and how they can be specified are described here in detail.

There are four classes of target specifications. For each target within the scene, the user can specify the: 

* spatial extent and morphology
* spectral energy distribution
* mapping of the velocity structure within the target
* line-of-sight velocity distribution 

These types of specifications are explained in detail here using the docstrings in the code itself to ensure that the instructions are as up-to-date as possible.  Each of the four types are specified in turn.

The way the types of sources are listed here (with single or double square brackets) shows how they should be specified in the scene.ini file (or other .ini file containing a sky scene).

In [1]:
%env PYSYN_CDBS=/Users/niklasviebig/Desktop/Python/JWST_semester_project/trds


env: PYSYN_CDBS=/Users/niklasviebig/Desktop/Python/JWST_semester_project/trds


In [2]:
# import the skysim component of MIRISim
# the component of MIRISim that generates scenes
import mirisim.skysim
outfile = 'valid_skysim_inputs.txt'

# enable printing to file, or to output screen
def print_opt(in_string,f=None):
    '''
    f: output file. If none is given, then output to screen
    '''
    print(str(in_string))
    if f:
        f.write(str(in_string))

#generic function to pull the documentation out of the **relevant** SkySim libraries
def get_documentation(module,blev,description,inputs):
    '''
    :Parameters:
    
    module: name of input  module to be explored for documentation
    blev:  level of brackets to output (consistent with .ini file notation)
    description: (True/False) should the description of the module be displayed 
    inputs: (True/False) should the required inputs be displayed
    '''
    #f = open(outfile,'w+')   #uncomment to output to file
    f = None                  #uncomment to output to screen
    comp_types = module.keys()
    for component in comp_types:
            target_lib = module[component]
            print_opt("{}{}{}".format('['*blev,component,']'*blev),f)
            if (description): print_opt(target_lib.__doc__,f)
            if (inputs): print(print_opt(target_lib.__init__.__doc__,f))
            print_opt("\n============================================\n",f)           

## What to display

There are two levels description available here. The first gives a description of the intent of the function (what kind of target or emission type is being created), and the second lists the available / required input parameters for that function.  

Whether the description or the inputs (or both) is displayed depends on which of the switches below are set to <code>True</code>

In [3]:
# Display function descriptions?
description = True
# Display input parameters?
inputs = True

## Specifying the Spatial Extent of the target


The primary classes of targets available within |skysim| are point and extended sources as well as a background radiation field. Both point and extended sources can be given spectral energy distribustions (SEDs), and spectral line information across the |MIRI| wavelength range, with extended sources having the additional capability of being encoded with information about an intrinsic velocity field. 

Extended sources:

*	have a spatial distribution of flux with is independent of wavelength
*	have an SED that gives the flux of the source at each wavelength.
*	*may* have a velocity map that gives the velocity along the line of sight at any given spatial coordinate (RA, DEC). This effectively "shifts" the SED in wavelength at each spatial coordinate.
*	*may* have a LOSVD that gives the shape (broadening and asymmetries1) due to kinematics of the spectral lines. The LOSVD is not variable with coordinates because that would be too computationally demanding and is perhaps not necessary for this kind of simulations.

Point sources:

*	have a position
*	have an SED, identical in implementation and capabilities to the SEDs of extended sources
	
** Special Case: **  Background emission (from the telescope and from the 'sky' are included in this specification. This is because the background has a spatial extent.


Background:

*	It implements the model given in [Glasse et al. 2015](http://iopscience.iop.org/article/10.1086/682259/pdf).
*	the background behaves, to the user, in analogous way to an extended source. It may be queried at any spatial coordinate and wavelength to retrieve an specific intensity
*	the JWST part of the background may be time and spatial dependent.
*	The spatial dependence is a linear gradient with a certain position angle. The gradient is fixed on JWST FOV coordinates (V2, V3).
*	Near Future: the Zodiacal component will vary depending on ecliptic coordinates of the observation (TBI).

These commands initialise what kind of source is going to be specified via the .ini file

#### Specification in scene.ini

These are the commands that go in single square brackets.  e.g.

<code> [Galaxy] </code>

#### Specifiication in python

These are the types of sources that initialise a target, eg:

<code> >> galaxy = Galaxy(Cen=(0.,0.),h=0.1,q=0.5,pa=35.,c=0) </code>
		
Where, `h` is the disk scale length, `q` is the axial ratio, `pa` is the in the position angle of the galaxy, and `c` represents its diskiness/boxiness. This galaxy is centred on the Principle Optical Path (POP).



----
Set  <code> Print_Docstrings = True </code> below to show which inputs are expected for specifying these targets.  Setting this parameter to <code>False</code> skips this step

In [4]:
Display_Docstrings = True

if (Display_Docstrings):
    get_documentation(mirisim.skysim.lib.source_lib,1,description,inputs)

[Background]

    Class for 'Sky + Observatory (JWST)' creates/returns background in [muJy/arcsec@]
    
 
        Initializes Background.
        
        Only JWST-part of the background may be spatial or time dependent.
        
        :Parameters:
        
        level: low / high.
        gradient: relative increase in JWST BGD over 1 arc-minute, 
                  given as a percentage.
                  Maximal allowed gradient is 10%.
        pa: [degree ; but stored in radian] position angle of the gradient (positive in the direction of pa).
            given in degrees, 0 along positive RA axis, 90 along positive DEC axis.
        tconstant: time-constant for time-dependent background [minutes].
        fast: Use 1 or not use 0 the 2.5D spee up. Speed up works for cases where flux(RA, DEC, WAV) = flux1(RA, DEC) * flux2(WAV)
        centreFOV: (RA, DEC) coordinates of the reference point (for gradient).
        
None


[Catalogue]
Class to read and parse a catalogue with sou

## Specifying the Spectral Energy Distribution

The spectral energy distribution (SED) of a target is specified using the facilities described below. 

There are four types of spectral energy distributions coded into SkySim: blackbody, stellar, galactic and planetary. 

** Special Case: ** Specification of spectral lines is included here since this involves specifying things in wavelength space.

#### Specifying in scene.ini

These specifications are presented in the .ini files with double square brackets.
Note that multiple spectral energy distributions (e.g. the combination of specific spectral lines with a blackbody) can be specified for a single target by copying the target properties to a second source with identical spatial properties, and a different SED.

<code>
[Galaxy]
params

[[sed]]
Type = SEDTYPE
params



</code>


#### Specifying within python

Using the galaxy object created above, an SED is added to the target using::

    galaxy.get_SED(sed.LinesSed(**SEDPars))
	
Where the `SEDPars` are a set of parameters defining, in this case, a spectral line SED with specific line fluxes at given wavelength with specific full width half max values. for example, the function in the brackets could take the form::

	sed.LinesSed(wavels=[5.5],fluxes=[150],fwhms=[300])

Multiple lines are then specified within the square brackets, and all three parameters must be specified for each line (wavelength, flux and fwhm).  

----
Set  <code> Print_Docstrings = True </code> below to show which inputs are expected for specifying these targets.  Setting this parameter to <code>False</code> skips this step

In [5]:
Display_Docstrings = True

if (Display_Docstrings):
    get_documentation(mirisim.skysim.lib.sed_lib,2,description,inputs)

[[BB]]
Black-Body Sed.

       :Parameters:

       Temp: Temperature [K]
       flux: Reference flux [muJy]
       wref: Reference wavelength of the reference spectrum [micron]
       
None
None


[[External]]
External Sed

       Loads and interpolates an External SED from a file

       :Paramaters:

       sedfile: (relative) path to the file containing columns 'wavelength' [microns] and 'flux' [muJy]
                Linear spacing is expceted.
                The file format must be compatible with astropy.io.ascii table
       wref: not used
       flux: not used
       

       :Usage in the scene.ini configuration file:

       [[sed]]
       Type = External
       sedfile = extsed.txt # filepath, units in [microns] [muJy]

        :Numpy array as input:
        
        When calling from a Python script you may omit `sedfile` parameter. You will get
        a warning after the init but then you can ignore it and load the numpy 2D array
        as an input. The 2D numpy array i

### Specifying Velocity Distributions



Galactic and protostellar disks alike rotate. SkySim gives you the ability to implement a velocity field into the extended sources created above. 


#### Specifying in scene.ini
<code>
[[velomap]]
   Type       = FlatDisk
   ...
</code>
#### Specifying in python

TBC



----
Set  <code> Print_Docstrings = True </code> below to show which inputs are expected for specifying these targets.  Setting this parameter to <code>False</code> skips this step

In [6]:
Display_Docstrings = True

if (Display_Docstrings):
    get_documentation(mirisim.skysim.lib.velomap_lib,2,description,inputs)

[[FlatDisk]]
Flat Rotation Curve Disk
 
None


[[KeplerDisk]]
Keplerian Disk
    
    
    v(r) = (v0 / r)^0.5
    
    
 
None




## Specifying line of sight velocity distribution


#### Specifying in scene.ini

<code>
[[losvd]]
    Type = ModGauss
    ...
</code>

#### Specifying in python

For instance, to add a line of sight velocity distribution (LOSVD) to the galaxy above::

	alosvd = kinetics.Losvd(200,h3=0,h4=0)
	galaxy.get_LOSVD(alosvd)
	
Where 200 is the velocity dispersion in km/s, and `h3` and `h4` are Gaussian Hermite polynomial coefficients.


----
For completeness, The equation for the line of sight velocity distribution is:

$$ losvd = \frac{1}{\sqrt{(2 \pi)}\sigma} * e^{-y^2/2} * (1 + h3*H3(y)+h4*H4(y))  $$


Set  <code> Print_Docstrings = True </code> below to show which inputs are expected for specifying these targets.  Setting this parameter to <code>False</code> skips this step


In [7]:
Print_Docstrings = True

if (Print_Docstrings):
    get_documentation(mirisim.skysim.lib.losvd_lib,2,description,inputs)

[[ModGauss]]
Line-of-Sight Velocity Distribution Class.
    
    Using a Gauss-Hermite expansion, after van der Marel & Franx 1993.
    
    

        :Parameters:
        
        sigma:
        h3: 
        h4: 
        
None




## 